## Preprocessing dan Feature Engineering

## Import Package

In [38]:
#Import library untuk data preparation dan visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import warnings for ignore the warnings
import warnings 
warnings.filterwarnings("ignore")

# import pickle and json file for columns and model file
import pickle
import json
import joblib
from sklearn.preprocessing import RobustScaler
import scipy.stats as scs
import src.util as util
import pickle
import os
import yaml
import pickle
import json
import joblib
import yaml
import scipy.stats as scs
import src.util as util
import pickle
import os

In [39]:
params_dir = "config/config.yaml"

In [40]:
def load_params(param_dir):
    with open(param_dir, 'r') as file:
        params = yaml.safe_load(file)
        
    return params

In [41]:
params = load_params(params_dir)

In [42]:
params

{'raw_dataset_dir': 'dataset/1 - raw data/',
 'train_set_path': ['dataset/2 - processed/X_train.pkl',
  'dataset/2 - processed/y_train.pkl'],
 'valid_set_path': ['dataset/2 - processed/X_valid.pkl',
  'dataset/2 - processed/y_valid.pkl'],
 'test_set_path': ['dataset/2 - processed/X_test.pkl',
  'dataset/2 - processed/y_test.pkl'],
 'robust_scaler_train': ['dataset/3 - final/X_train_robust.pkl',
  'dataset/3 - final/y_train.pkl'],
 'robust_scaler_valid': ['dataset/3 - final/X_valid_robust.pkl',
  'dataset/3 - final/y_valid.pkl'],
 'robust_scaler_test': ['dataset/3 - final/X_test_robust.pkl',
  'dataset/3 - final/y_test.pkl'],
 'production_model_path': 'model/5 - Model Final',
 'model_robust_scaler': 'model/5 - Model Final/robust_scaler.pkl',
 'robust_scaler': 'model/5 - Model Final/',
 'model_final': 'model/5 - Model Final/xgboost_cv_robust.pkl',
 'print_debug': True,
 'int64_columns': ['Year', 'Kms_Driven', 'Owner'],
 'float64_columns': ['Present_Price'],
 'object_columns': ['Fuel_Type

In [43]:
config_data = util.load_config()

## Read Data

In [7]:
"""
#Read data X_train dan y_train hasil dari data preparation
X_train = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_train.pkl")
y_train = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_train.pkl")

#Read data X_valid dan y_valid hasil dari data preparation
X_valid = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_valid.pkl")
y_valid = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_valid.pkl")

#Read data X_test dan y_test hasil dari data preparation
X_test = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_test.pkl")
y_test = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_test.pkl")
"""

'\n#Read data X_train dan y_train hasil dari data preparation\nX_train = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_train.pkl")\ny_train = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_train.pkl")\n\n#Read data X_valid dan y_valid hasil dari data preparation\nX_valid = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_valid.pkl")\ny_valid = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_valid.pkl")\n\n#Read data X_test dan y_test hasil dari data preparation\nX_test = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\X_test.pkl")\ny_test = joblib.load("C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\2 - processed\\y_test.pkl")\n'

In [8]:
X_train = util.pickle_load(config_data["train_set_path"][0])
y_train = util.pickle_load(config_data["train_set_path"][1])

X_valid = util.pickle_load(config_data["valid_set_path"][0])
y_valid = util.pickle_load(config_data["valid_set_path"][1])

X_test = util.pickle_load(config_data["test_set_path"][0])
y_test = util.pickle_load(config_data["test_set_path"][1])

In [9]:
#Menyatukan data X_train dan y_train untuk dilakukan preprocessing dan Feature Engineering menjadi satu set
dataset = pd.concat([X_train, y_train], axis = 1)

#Menyatukan data X_valid dan y_valid untuk dilakukan preprocessing dan Feature Engineering menjadi satu set
valid_set = pd.concat([X_valid, y_valid], axis = 1)

#Menyatukan data X_test dan y_test untuk dilakukan preprocessing dan Feature Engineering menjadi satu set
test_set = pd.concat([X_test, y_test], axis = 1)

In [10]:
dataset

,Year,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Selling_Price
5,2018,9.83,2071,Diesel,Dealer,Manual,0,9.25
258,2015,13.60,25000,Petrol,Dealer,Manual,0,8.40
278,2016,8.40,4000,Petrol,Dealer,Manual,0,6.00
260,2016,13.60,29223,Petrol,Dealer,Manual,0,9.15
7,2015,8.61,33429,Diesel,Dealer,Manual,0,6.50
...,...,...,...,...,...,...,...,...
106,2014,3.45,16500,Petrol,Individual,Manual,1,1.35
83,2015,13.46,38000,Diesel,Dealer,Manual,0,12.50
17,2016,10.79,43000,Diesel,Dealer,Manual,0,7.75
230,2013,9.40,45000,Diesel,Dealer,Manual,0,6.15


## Handling Missing Value

In [14]:
from sklearn.impute import SimpleImputer

def imputeData(data, numerical_columns_modus, numerical_columns_median, categorical_columns):
    """
    Fungsi untuk melakukan imputasi data numerik dan kategorikal
    :param data: <pandas dataframe> sample data input
    :param numerical_columns_modus: <list> list kolom numerik data yang akan diimputasi dengan modus
    :param numerical_columns_median: <list> list kolom numerik data yang akan diimputasi dengan median
    :param categorical_columns: <list> list kolom kategorikal data
    :return numerical_data_imputed: <pandas dataframe> data numerik imputed
    :return categorical_data_imputed: <pandas dataframe> data kategorikal imputed
    """
    # Imputasi kolom numerik dengan modus
    numerical_data_modus = data[numerical_columns_modus]
    imputer_numerical_modus = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputer_numerical_modus.fit(numerical_data_modus)
    imputed_data_modus = imputer_numerical_modus.transform(numerical_data_modus)
    numerical_data_imputed_modus = pd.DataFrame(imputed_data_modus, columns=numerical_columns_modus, index=numerical_data_modus.index)

    # Imputasi kolom numerik dengan median
    numerical_data_median = data[numerical_columns_median]
    imputer_numerical_median = SimpleImputer(missing_values=np.nan, strategy='median')
    imputer_numerical_median.fit(numerical_data_median)
    imputed_data_median = imputer_numerical_median.transform(numerical_data_median)
    numerical_data_imputed_median = pd.DataFrame(imputed_data_median, columns=numerical_columns_median, index=numerical_data_median.index)

    # Gabungkan kedua data numerik yang telah diimputasi
    numerical_data_imputed = pd.concat([numerical_data_imputed_modus, numerical_data_imputed_median], axis=1)

    # Seleksi data kategorikal
    categorical_data = data[categorical_columns]

    # Imputasi dengan menggunakan modus
    mode = categorical_data.mode().iloc[0]

    # Lakukan imputasi untuk data kategorikal
    categorical_data_imputed = categorical_data.fillna(mode)

    # Gabungkan data numerik dan kategorikal yang telah diimputasi
    data_imputed = pd.concat([numerical_data_imputed, categorical_data_imputed], axis=1)

    return data_imputed

## Parameter Imputasi

In [15]:
# Buat kolom numerik
numerical_columns_median = ["Kms_Driven", "Present_Price", "Owner"]
numerical_columns_modus = ["Year"]
set_numerik = numerical_columns_median + numerical_columns_modus
dataset_column = list(X_train.columns)
categorical_column = list(set(dataset_column).difference(set(set_numerik)))

# Imputasi tabel dataset (train)

In [16]:
X_train_impute = imputeData(data = X_train, 
                            numerical_columns_modus = numerical_columns_modus, 
                            numerical_columns_median = numerical_columns_median, 
                            categorical_columns = categorical_column)

In [17]:
X_train_impute

,Year,Kms_Driven,Present_Price,Owner,Seller_Type,Fuel_Type,Transmission
5,2018,2071.0,9.83,0.0,Dealer,Diesel,Manual
258,2015,25000.0,13.60,0.0,Dealer,Petrol,Manual
278,2016,4000.0,8.40,0.0,Dealer,Petrol,Manual
260,2016,29223.0,13.60,0.0,Dealer,Petrol,Manual
7,2015,33429.0,8.61,0.0,Dealer,Diesel,Manual
...,...,...,...,...,...,...,...
106,2014,16500.0,3.45,1.0,Individual,Petrol,Manual
83,2015,38000.0,13.46,0.0,Dealer,Diesel,Manual
17,2016,43000.0,10.79,0.0,Dealer,Diesel,Manual
230,2013,45000.0,9.40,0.0,Dealer,Diesel,Manual


## Imputasi tabel valid_set

In [18]:
X_valid_impute = imputeData(data = X_valid, 
                            numerical_columns_modus = numerical_columns_modus, 
                            numerical_columns_median = numerical_columns_median, 
                            categorical_columns = categorical_column)

In [19]:
X_valid_impute

,Year,Kms_Driven,Present_Price,Owner,Seller_Type,Fuel_Type,Transmission
20,2016,25000.0,3.950,0.0,Dealer,Petrol,Manual
105,2015,21700.0,2.370,0.0,Individual,Petrol,Manual
174,2015,38600.0,0.720,0.0,Individual,Petrol,Manual
294,2014,33019.0,6.800,0.0,Dealer,Petrol,Manual
293,2010,38000.0,9.900,0.0,Dealer,Petrol,Manual
160,2011,24000.0,0.950,0.0,Individual,Petrol,Manual
19,2010,41442.0,7.980,0.0,Dealer,Petrol,Manual
102,2017,4000.0,1.780,0.0,Individual,Petrol,Manual
241,2015,35866.0,7.130,1.0,Dealer,Petrol,Manual
210,2012,35775.0,4.600,0.0,Dealer,Petrol,Manual


## Imputasi tabel test_set

In [20]:
X_test_impute = imputeData(data = X_test, 
                            numerical_columns_modus = numerical_columns_modus, 
                            numerical_columns_median = numerical_columns_median, 
                            categorical_columns = categorical_column)

In [21]:
X_test_impute

,Year,Kms_Driven,Present_Price,Owner,Seller_Type,Fuel_Type,Transmission
256,2016,49562.0,13.600,0.0,Dealer,Petrol,Manual
26,2013,55138.0,5.870,0.0,Dealer,Petrol,Manual
215,2012,36100.0,9.400,0.0,Dealer,Petrol,Manual
150,2011,6000.0,0.826,0.0,Individual,Petrol,Manual
148,2010,45000.0,0.940,0.0,Individual,Petrol,Manual
268,2017,19000.0,5.800,0.0,Dealer,Petrol,Manual
82,2017,15000.0,25.390,0.0,Dealer,Diesel,Automatic
195,2015,35000.0,0.320,0.0,Individual,Petrol,Manual
24,2013,56879.0,4.410,0.0,Dealer,Petrol,Manual
218,2014,45078.0,9.400,0.0,Dealer,Petrol,Manual


## Get Dummies

In [22]:
def get_dummies(train_df, input_df):
    # Menggabungkan data train dan input menjadi satu DataFrame
    combined_df = pd.concat([train_df, input_df])
    
    # Mengubah variabel kategorikal menjadi variabel dummy
    dummies_df = pd.get_dummies(combined_df, columns=train_df.select_dtypes(include='object').columns)
    
    # Memisahkan kembali data train dan input
    train_dummies = dummies_df[:train_df.shape[0]]
    input_dummies = dummies_df[train_df.shape[0]:]
    
    return train_dummies, input_dummies

In [23]:
dataset_ohe, valid_set = get_dummies(X_train_impute, X_valid_impute)

In [24]:
dataset, test_set = get_dummies(X_train_impute, X_test_impute)

In [25]:
dataset

,Year,Kms_Driven,Present_Price,Owner,Seller_Type_Dealer,Seller_Type_Individual,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
5,2018,2071.0,9.83,0.0,1,0,0,1,0,0,1
258,2015,25000.0,13.60,0.0,1,0,0,0,1,0,1
278,2016,4000.0,8.40,0.0,1,0,0,0,1,0,1
260,2016,29223.0,13.60,0.0,1,0,0,0,1,0,1
7,2015,33429.0,8.61,0.0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
106,2014,16500.0,3.45,1.0,0,1,0,0,1,0,1
83,2015,38000.0,13.46,0.0,1,0,0,1,0,0,1
17,2016,43000.0,10.79,0.0,1,0,0,1,0,0,1
230,2013,45000.0,9.40,0.0,1,0,0,1,0,0,1


In [26]:
test_set

,Year,Kms_Driven,Present_Price,Owner,Seller_Type_Dealer,Seller_Type_Individual,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
256,2016,49562.0,13.600,0.0,1,0,0,0,1,0,1
26,2013,55138.0,5.870,0.0,1,0,0,0,1,0,1
215,2012,36100.0,9.400,0.0,1,0,0,0,1,0,1
150,2011,6000.0,0.826,0.0,0,1,0,0,1,0,1
148,2010,45000.0,0.940,0.0,0,1,0,0,1,0,1
268,2017,19000.0,5.800,0.0,1,0,0,0,1,0,1
82,2017,15000.0,25.390,0.0,1,0,0,1,0,1,0
195,2015,35000.0,0.320,0.0,0,1,0,0,1,0,1
24,2013,56879.0,4.410,0.0,1,0,0,0,1,0,1
218,2014,45078.0,9.400,0.0,1,0,0,0,1,0,1


In [27]:
valid_set

,Year,Kms_Driven,Present_Price,Owner,Seller_Type_Dealer,Seller_Type_Individual,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual
20,2016,25000.0,3.950,0.0,1,0,0,0,1,0,1
105,2015,21700.0,2.370,0.0,0,1,0,0,1,0,1
174,2015,38600.0,0.720,0.0,0,1,0,0,1,0,1
294,2014,33019.0,6.800,0.0,1,0,0,0,1,0,1
293,2010,38000.0,9.900,0.0,1,0,0,0,1,0,1
160,2011,24000.0,0.950,0.0,0,1,0,0,1,0,1
19,2010,41442.0,7.980,0.0,1,0,0,0,1,0,1
102,2017,4000.0,1.780,0.0,0,1,0,0,1,0,1
241,2015,35866.0,7.130,1.0,1,0,0,0,1,0,1
210,2012,35775.0,4.600,0.0,1,0,0,0,1,0,1


## RobustScaler

In [28]:
from sklearn.preprocessing import RobustScaler
import pickle
import os

columns_to_scale =  ["Kms_Driven","Present_Price"]

def fit_scaler(train_data):
    scaler = RobustScaler()
    scaler.fit(train_data.loc[:, columns_to_scale])
    # save scaler
    with open('model/5 - Model Final/robust_scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    return scaler

def load_scaler(folder_path):
    # load scaler
    file_path = os.path.join(folder_path, 'robust_scaler.pkl')
    with open(file_path, 'rb') as f:
        scaler = pickle.load(f)
    return scaler

def transform_data(data, scaler):
    scaled_data = scaler.transform(data.loc[:, columns_to_scale])
    data.loc[:, columns_to_scale] = scaled_data
    return data

In [29]:
robust_scaler = fit_scaler(dataset)
robust_load = load_scaler('model/5 - Model Final/')

In [30]:
X_train_robust = transform_data(dataset, robust_load)
X_valid_robust = transform_data(valid_set, robust_load)
X_test_robust = transform_data(test_set, robust_load)

In [31]:
X_train_robust = X_train_robust[sorted(X_train_robust.columns)]
X_valid_robust = X_valid_robust[sorted(X_valid_robust.columns)]
X_test_robust = X_test_robust[sorted(X_test_robust.columns)]

In [33]:
X_train_robust

,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Kms_Driven,Owner,Present_Price,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual,Year
5,0,1,0,-0.859354,0.0,0.434483,1,0,0,1,2018
258,0,0,1,-0.153846,0.0,0.867816,1,0,0,1,2015
278,0,0,1,-0.800000,0.0,0.270115,1,0,0,1,2016
260,0,0,1,-0.023908,0.0,0.867816,1,0,0,1,2016
7,0,1,0,0.105508,0.0,0.294253,1,0,0,1,2015
...,...,...,...,...,...,...,...,...,...,...,...
106,0,0,1,-0.415385,1.0,-0.298851,0,1,0,1,2014
83,0,1,0,0.246154,0.0,0.851724,1,0,0,1,2015
17,0,1,0,0.400000,0.0,0.544828,1,0,0,1,2016
230,0,1,0,0.461538,0.0,0.385057,1,0,0,1,2013


In [34]:
X_valid_robust

,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Kms_Driven,Owner,Present_Price,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual,Year
20,0,0,1,-0.153846,0.0,-0.241379,1,0,0,1,2016
105,0,0,1,-0.255385,0.0,-0.422989,0,1,0,1,2015
174,0,0,1,0.264615,0.0,-0.612644,0,1,0,1,2015
294,0,0,1,0.092892,0.0,0.086207,1,0,0,1,2014
293,0,0,1,0.246154,0.0,0.442529,1,0,0,1,2010
160,0,0,1,-0.184615,0.0,-0.586207,0,1,0,1,2011
19,0,0,1,0.352062,0.0,0.221839,1,0,0,1,2010
102,0,0,1,-0.800000,0.0,-0.490805,0,1,0,1,2017
241,0,0,1,0.180492,1.0,0.124138,1,0,0,1,2015
210,0,0,1,0.177692,0.0,-0.166667,1,0,0,1,2012


In [35]:
X_test_robust

,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Kms_Driven,Owner,Present_Price,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual,Year
256,0,0,1,0.601908,0.0,0.867816,1,0,0,1,2016
26,0,0,1,0.773477,0.0,-0.020690,1,0,0,1,2013
215,0,0,1,0.187692,0.0,0.385057,1,0,0,1,2012
150,0,0,1,-0.738462,0.0,-0.600460,0,1,0,1,2011
148,0,0,1,0.461538,0.0,-0.587356,0,1,0,1,2010
268,0,0,1,-0.338462,0.0,-0.028736,1,0,0,1,2017
82,0,1,0,-0.461538,0.0,2.222989,1,0,1,0,2017
195,0,0,1,0.153846,0.0,-0.658621,0,1,0,1,2015
24,0,0,1,0.827046,0.0,-0.188506,1,0,0,1,2013
218,0,0,1,0.463938,0.0,0.385057,1,0,0,1,2014


In [36]:
y_test

256    10.25
26      4.15
215     4.50
150     0.50
148     0.52
268     4.80
82     23.00
195     0.18
24      2.90
218     6.45
204     2.75
170     0.40
138     0.60
279     6.25
71      4.50
52     18.00
282     8.25
297     4.00
266     3.65
267     8.35
124     0.95
144     0.60
35      2.95
250    12.90
80     14.73
22      4.40
54      2.75
173     0.40
72      7.45
81      4.75
21      4.90
259     3.90
136     0.65
41      2.55
166     0.45
290     4.50
235     9.10
89      4.75
137     0.65
289    10.11
249     5.25
286     5.65
179     0.31
75      3.95
222     6.00
Name: Selling_Price, dtype: float64

## Save Data

In [37]:
util.pickle_dump(X_train_robust, config_data["robust_scaler_train"][0])
util.pickle_dump(y_train, config_data["robust_scaler_train"][1])
    
util.pickle_dump(X_test_robust, config_data["robust_scaler_test"][0])
util.pickle_dump(y_test, config_data["robust_scaler_test"][1])

util.pickle_dump(X_valid_robust, config_data["robust_scaler_valid"][0])
util.pickle_dump(y_valid, config_data["robust_scaler_valid"][1])

In [45]:
# Original Data
joblib.dump(train_set_ori, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\train_set_ori.pkl")
joblib.dump(valid_set_ori, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\valid_set_ori.pkl")
joblib.dump(test_set_ori, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\test_set_ori.pkl")
joblib.dump(y_train, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\y_train.pkl")
joblib.dump(y_valid, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\y_valid.pkl")
joblib.dump(y_test, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\y_test.pkl")

# StandartScaler
joblib.dump(X_train_scaler, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_train_scaler.pkl")
joblib.dump(X_valid_scaler, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_valid_scaler.pkl")
joblib.dump(X_test_scaler, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_test_scaler.pkl")

# MinMaxScaler
joblib.dump(X_train_minmax, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_train_minmax.pkl")
joblib.dump(X_valid_minmax, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_valid_minmax.pkl")
joblib.dump(X_test_minmax, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_test_minmax.pkl")

# RobustScaler
joblib.dump(X_train_robust, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_train_robust.pkl")
joblib.dump(X_valid_robust, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_valid_robust.pkl")
joblib.dump(X_test_robust, "C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_test_robust.pkl")

['C:\\Users\\hp\\Portofolio Data Science\\4 - Vehicle Price Prediction\\dataset\\3 - final\\X_test_robust.pkl']